# Miltiades Vasiliades 2944
## Second Set Data Mining
### Winter 2020

In [1]:
import scipy as sp
import numpy as np 
import pandas as pd 
import scipy.sparse as sp_sparse
import scipy.spatial.distance as sp_dist

import matplotlib.pyplot as plt

import sklearn as sk
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn import preprocessing
import sklearn.cluster as sk_cluster
import sklearn.feature_extraction.text as sk_text

import scipy.cluster.hierarchy as hr

import seaborn as sns

#import json
#import ijson
import time


In [2]:
businesses = pd.read_json(path_or_buf='yelp_academic_dataset_business.zip',lines=True,compression='infer')
businesses = businesses[businesses['city']=='Toronto']
businesses = businesses['business_id'].to_frame()

#display(businesses)

In [3]:
businesses.to_csv('businessToronto.csv',index=False)

In [6]:
pruned = pd.read_csv('pruned_data.csv',header=0,names=['user_id','business_id','rating'])
pruned.head()

,user_id,business_id,rating
0,TZQSUDDcA4ek5gBd6BzcjA,6n_MDeYxU1ihB38be9TkVA,4.0
1,TZQSUDDcA4ek5gBd6BzcjA,jo4KmAqlZ7vxjHIP7IIkAw,5.0
2,TZQSUDDcA4ek5gBd6BzcjA,dsAcgF6qtZy2m6d_yWCrGQ,4.0
3,TZQSUDDcA4ek5gBd6BzcjA,siaRCT2-PkyeXUVKrywcTg,4.0
4,TZQSUDDcA4ek5gBd6BzcjA,SGP1jf6k7spXkgwBlhiUVw,5.0


In [69]:
R = pruned.pivot_table(index='user_id',columns='business_id',values='rating')
R.head()

business_id,--SrzpvFLwP_YFwB_Cetow,-0NhdsDJsdarxyDPR523ZQ,-2TBP3ZGu7M-FmfoNJvbrQ,-2hOglg7Lh8ZgclQJ9ba2w,-6GJMc_Ln7csmvWWvY2iug,-76didnxGiiMO80BjSpYsQ,-7BCZH437U5FjmNJ26llkg,-9dmhyBvepc08KPEHlEM0w,-9u2uFwhlj3Yq4mG5IweoA,-AVRReI-nfsa0lKlehEojw,...,zpwl8rQBGrO1XkP1CYBtvg,zqPAwZbi5rS1dyHrKzDpMg,zqjcp1VwY8WNtiwTvFL1Rw,zrYpLdnGKA_EmOhgRCy_vg,zu5dqKqPjJCycS5k-92Alg,zvezLicNkf8tTqZyYhP83w,zvtkeghW0Px5HY9QkJ4INw,zyw5DjrRks7a8OhmBsgCQQ,zz3CqZhNx2rQ_Yp6zHze-A,zzUj3ej4vm_DtvRxNvWDEw
user_id,,,,,,,,,,,,,,,,,,,,,
--BumyUHiO_7YsHurb9Hkw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Qh8yKWAvIP4V4K8ZPfHA,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
--UOvCH5qEgdNQ8lzR8QYQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--YhjyV-ce1nFLYxP49C5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--cd_gA-9Q8gM9P2cTxEsQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
#seed = round(time.time())
seed = 10
random_st = np.random.seed(seed)
to_be_Zeroed = round(len(R.index)*0.05)
random_ratings = R.sample(frac=0.05,replace=False, random_state=random_st,axis=0)
random_sample_index = np.random.choice(R.index, to_be_Zeroed, replace=False)
#R.replace(to_replace=random_sample_index,value=None,inplace=True)
R.loc[random_sample_index]=np.nan
#R.head()
#display(random_ratings)

In [75]:

R = R.to_numpy(na_value=0)
random_ratings = random_ratings.to_numpy(na_value=0)

In [76]:

distances = metrics.pairwise.pairwise_distances(R,metric = 'euclidean',force_all_finite='allow-nan')
display(distances)

array([[ 0.        , 58.33523806, 35.73513677, ..., 34.10278581,
        31.30495168, 33.45145737],
       [58.33523806,  0.        , 58.94064811, ..., 56.93856338,
        55.57877293, 57.14892825],
       [35.73513677, 58.94064811,  0.        , ..., 33.7934905 ,
        30.96772513, 32.52691193],
       ...,
       [34.10278581, 56.93856338, 33.7934905 , ...,  0.        ,
        28.37252192, 30.72458299],
       [31.30495168, 55.57877293, 30.96772513, ..., 28.37252192,
         0.        , 27.03701167],
       [33.45145737, 57.14892825, 32.52691193, ..., 30.72458299,
        27.03701167,  0.        ]])